In [1]:
#importing libraries
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize
import re
from nltk.stem import PorterStemmer
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from keras.models import Sequential
from keras.layers import Dense
from sknn.mlp import Classifier, Layer
from sklearn.model_selection import train_test_split


# fix random seed for reproducibility
np.random.seed(7)

Using TensorFlow backend.


ImportError: No module named 'tensorflow'

In [4]:
def title_cleaner(title):
    title = re.sub('[^a-zA-Z]',' ', title)
    title = title.lower()
    title = nltk.word_tokenize(title) 
    eng_stopwords = set(stopwords.words("english"))
    title = [w for w in title if not w in eng_stopwords]
    title = ' '.join([word for word in title])
    return(title)

wnl = WordNetLemmatizer()
def get_wordnet_pos(treebank_tag):
    '''Treebank to wordnet POS tag'''
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return 'n' #basecase POS

In [5]:
#reading the file
data = pd.read_csv('fake.csv')

In [6]:
del data['uuid']

In [7]:
data = data.dropna().reset_index()

In [8]:
del data['index']

In [9]:
positive = data[data['spam_score'] < 0.5]
negative = data[data['spam_score'] >= 0.5]
no_of_positive = len(positive)
no_of_negative = len(negative)
print ("Number of not spam news: ", no_of_positive)
print ("Number of spam news: :", no_of_negative)
print ("Percentage of spam vs ham:" ,no_of_negative*100/no_of_positive)

('Number of not spam news: ', 4656)
('Number of spam news: :', 46)
('Percentage of spam vs ham:', 0)


In [10]:
result = [0 if val < 0.5 else 1 for val in data['spam_score']]


In [11]:
len_of_reviews = np.array([])
for val in data['title']:
    temp = len(val)
    len_of_reviews = np.append(len_of_reviews,temp)

print ("The average length of all the reviews: ",len_of_reviews.max())

('The average length of all the reviews: ', 283.0)


In [12]:
num_title = len(data['title'])
title_clean_original = []
for i in range(0,num_title - 1):
    if( (i+1)%500 == 0 ):
         # print progress
        print("Done with %d title for title_clean_original" %(i+1)) 
    title_clean_original.append(title_cleaner(data['title'][i]))
    
num_text = len(data['text'])
text_clean_original = []
for i in range(0,num_text - 1):
    if( (i+1)%500 == 0 ):
         # print progress
        print("Done with %d title for title_clean_original" %(i+1)) 
    text_clean_original.append(title_cleaner(data['text'][i]))
        

Done with 500 title for title_clean_original
Done with 1000 title for title_clean_original
Done with 1500 title for title_clean_original
Done with 2000 title for title_clean_original
Done with 2500 title for title_clean_original
Done with 3000 title for title_clean_original
Done with 3500 title for title_clean_original
Done with 4000 title for title_clean_original
Done with 4500 title for title_clean_original
Done with 500 title for title_clean_original
Done with 1000 title for title_clean_original
Done with 1500 title for title_clean_original
Done with 2000 title for title_clean_original
Done with 2500 title for title_clean_original
Done with 3000 title for title_clean_original
Done with 3500 title for title_clean_original
Done with 4000 title for title_clean_original
Done with 4500 title for title_clean_original


In [13]:
title_clean_wnl = []
for val in title_clean_original:
    l = pos_tag(val.split())
    temp = ' '.join([wnl.lemmatize(w,pos=get_wordnet_pos(t)) for w,t in l])
    title_clean_wnl.append(temp)
    
text_clean_wnl = []
for val in text_clean_original:
    l = pos_tag(val.split())
    temp = ' '.join([wnl.lemmatize(w,pos=get_wordnet_pos(t)) for w,t in l])
    text_clean_wnl.append(temp)

KeyboardInterrupt: 

In [ ]:
vectorizer = CountVectorizer(analyzer = "word",   \
                                 tokenizer = None,    \
                                 preprocessor = None, \
                                 stop_words = None,   \
                                 max_features = 277) 

In [ ]:
df = pd.DataFrame(vectorizer.fit_transform(data['title']).toarray())
df1 = pd.DataFrame(vectorizer.fit_transform(data['title']).toarray())

In [ ]:
train_data = pd.concat([data,df,df1], axis = 1)

In [ ]:
del train_data['title']

In [ ]:
del train_data['thread_title']

In [ ]:
del train_data['text']

In [ ]:
del train_data['type']
del train_data['spam_score']

In [ ]:
del train_data['main_img_url']

In [ ]:
del train_data['published']
del train_data['crawled']

In [ ]:
l = ['country','site_url','author','language']
le=LabelEncoder()
for col in l:
    le.fit(train_data[col])
    train_data[col]=le.transform(train_data[col])

In [ ]:
y_train = np.transpose(np.array([result]))

In [127]:
X_train, X_test, y_train, y_test = train_test_split(train_data, y_train, test_size=0.30, random_state=42)

In [129]:
forest = RandomForestClassifier(n_estimators = 50) 
forest = forest.fit(X_train, y_train)

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  from ipykernel import kernelapp as app


In [132]:
prediction = forest.predict(X_test)
accuracy  = metrics.accuracy_score(y_test,prediction)
accuracy

0.99007795889440109

In [139]:
# create model
model = Sequential()
model.add(Dense(600, input_dim=565, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [140]:
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [141]:
model.fit(X_train, y_train)

Epoch 1/10


InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [133]:
nn = Classifier(
    layers=[
        Layer("Rectifier", units=600),
        Layer("Softmax")],
    learning_rate=0.02)

In [150]:
X_train = X_train.reset_index()
X_test = X_test.reset_index()
del X_train['index']
del X_test['index']

In [148]:
nn.fit(f, y_train)

Classifier(batch_size=1, callback=None, debug=False, dropout_rate=None,
      f_stable=0.001,
      hidden0=<sknn.nn.Layer `Rectifier`: units=600, name=u'hidden0', frozen=False>,
      layers=[<sknn.nn.Layer `Rectifier`: units=600, name=u'hidden0', frozen=False>, <sknn.nn.Layer `Softmax`: units=2, name=u'output', frozen=False>],
      learning_momentum=0.9, learning_rate=0.02, learning_rule=u'sgd',
      loss_type=None, n_iter=None, n_stable=10, normalize=None,
      output=<sknn.nn.Layer `Softmax`: units=2, name=u'output', frozen=False>,
      parameters=None, random_state=None, regularize=None, valid_set=None,
      valid_size=0.0, verbose=None, warning=None, weight_decay=None)

In [151]:
prediction = nn.predict(X_test)
accuracy  = metrics.accuracy_score(y_test,prediction)
accuracy

[(1411, 2)]


0.99149539333805814